<a href="https://colab.research.google.com/github/alt159/public_works/blob/main/DataExtractorBCRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import re  # Importar el módulo de expresiones regulares

# Definir los parámetros de la consulta
url_base = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api"
codigo_serie = "PN01728AM"
formato_salida = "json"
periodo_inicial = "2004-01"
periodo_final = "2023-06"
idioma = "es"

# Construir la URL completa
url = f"{url_base}/{codigo_serie}/{formato_salida}/{periodo_inicial}/{periodo_final}/{idioma}"

# Hacer la solicitud GET a la API
response = requests.get(url)

# Imprimir el código de estado y el tipo de contenido de la respuesta
print("Código de Estado:", response.status_code)
print("Tipo de Contenido:", response.headers.get('Content-Type'))

# Verificar el tipo de contenido antes de decodificar como JSON
if 'application/json' in response.headers.get('Content-Type', ''):
    try:
        # Intentar decodificar la respuesta como JSON y convertir a DataFrame
        datos = response.json()
        df = pd.DataFrame(datos)
        print("Datos convertidos a DataFrame:")
        print(df)
    except Exception as e:
        print("Error al decodificar JSON:", str(e))
        print("Contenido de la Respuesta:", response.text)
else:
    print("La respuesta no es un JSON.")
    print("Contenido de la Respuesta:", response.text)

    # Definir un patrón para extraer el "name" y "values" del contenido
    pattern = re.compile(r'"name":"([^"]+)",\s*"values":\["([^"]+)"\]')

    # Encontrar todas las coincidencias del patrón en el contenido
    matches = pattern.findall(response.text)

    # Preparar una lista para contener los datos
    data = []

    # Iterar sobre las coincidencias y añadir los datos extraídos a la lista
    for name, value in matches:
        try:
            # Intentar convertir el valor a float y añadirlo a la lista de datos
            data.append({"name": name, "value": float(value)})
        except ValueError:
            # Si la conversión falla, añadir el valor tal como está
            data.append({"name": name, "value": value})

    # Convertir la lista de diccionarios en un DataFrame
    df_extraida = pd.DataFrame(data)

    # Mostrar las primeras filas del DataFrame extraído
    print("Datos extraídos convertidos a DataFrame:")
    print(df_extraida.head())
